# MNIST GPU Deep Learning Benchmark with H2O Deep Water
In reference to [Szilard's Benchmark-DL](https://github.com/szilard/benchm-dl)

In [23]:
import sys, os
import os.path
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
PATH = os.path.expanduser("~/h2o-3/")
h2o.init(nthreads=-1)
if not H2ODeepWaterEstimator.available(): exit

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,8 mins 25 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,10 hours and 16 minutes
H2O cluster name:,H2O_from_python_arno_mc0sih
H2O cluster total nodes:,1
H2O cluster free memory:,13.46 Gb
H2O cluster total cores:,40
H2O cluster allowed cores:,40
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


### We use one NVidia GTX1080

In [25]:
!nvidia-smi

Mon Oct 24 09:49:53 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.44                 Driver Version: 367.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0      On |                  N/A |
| 27%   36C    P8    10W / 180W |   1746MiB /  8097MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 0000:81:00.0      On |                  N/A |
| 27%   40C    P8    10W / 180W |   1201MiB /  8113MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
        

### We define the CNN

In [17]:
def cnn(num_classes):
    import mxnet as mx
    data = mx.symbol.Variable('data')

    conv1 = mx.symbol.Convolution(data=data, kernel=(4,4), num_filter=32)
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    pool1 = mx.symbol.Pooling(data=relu1, pool_type="max", kernel=(2,2), stride=(2,2))

    conv2 = mx.symbol.Convolution(data=pool1, kernel=(3,3), num_filter=16)
    relu2 = mx.symbol.Activation(data=conv2, act_type="relu")
    pool2 = mx.symbol.Pooling(data=relu2, pool_type="max", kernel=(2,2), stride=(2,2))
    drop = mx.symbol.Dropout(data=pool2, p=0.2)

    flatten = mx.symbol.Flatten(data=drop)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=128)
    relu3 = mx.symbol.Activation(data=fc1, act_type="relu")

    fc2 = mx.symbol.FullyConnected(data=relu3, num_hidden=64)
    relu4 = mx.symbol.Activation(data=fc2, act_type="relu")

    fc3 = mx.symbol.FullyConnected(data=relu4, num_hidden=num_classes)
    net = mx.symbol.SoftmaxOutput(data=fc3, name='softmax')
    return net

### Let's train the model

In [26]:
train = h2o.import_file(PATH + "bigdata/laptop/mnist/train.csv.gz")
predictors  = list(range(0,784))
resp        = 784
train[resp] = train[resp].asfactor()
nclasses    = train[resp].nlevels()[0]
cnn(nclasses).save("/tmp/cnn.json")
model = H2ODeepWaterEstimator(epochs=10,
                              learning_rate=0.05,
                              learning_rate_annealing=1e-5,
                              momentum_start=0.9,
                              momentum_stable=0.9,
                              mini_batch_size=128,
                              train_samples_per_iteration=0,
                              score_duty_cycle=0,
                              stopping_rounds=0,
                              ignore_const_cols=False,
                              network_definition_file="/tmp/cnn.json",
                              image_shape=[28,28],
                              channels=1,
                              device_id=[0])

model.train(x=predictors,y=resp, training_frame=train)

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%
deepwater Model Build progress: |█████████████████████████████████████████████████████████████| 100%


### It takes about 26 seconds to train 600k samples

In [27]:
model.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error
0,,2016-10-24 09:50:41,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN
1,,2016-10-24 09:51:07,26.250 sec,23499 obs/sec,10.005333,10,600320.0,0.061987,0.014723,0.004052


### Let's evaluate the test set performance

In [28]:
test = h2o.import_file(PATH + "bigdata/laptop/mnist/test.csv.gz")
print(model.model_performance(test))

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%

ModelMetricsMultinomial: deepwater
** Reported on test data. **

MSE: 0.00895735071088
RMSE: 0.0946432813827
LogLoss: 0.0334264931925
Mean Per-Class Error: 0.01115655357
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
974.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0061224,6 / 980
0.0,1133.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0017621,"2 / 1,135"
0.0,1.0,1019.0,3.0,1.0,0.0,1.0,5.0,2.0,0.0,0.0125969,"13 / 1,032"
0.0,0.0,0.0,1003.0,0.0,3.0,0.0,2.0,2.0,0.0,0.0069307,"7 / 1,010"
0.0,0.0,1.0,0.0,975.0,0.0,2.0,0.0,2.0,2.0,0.0071283,7 / 982
0.0,0.0,0.0,8.0,0.0,881.0,0.0,0.0,1.0,2.0,0.0123318,11 / 892
6.0,3.0,2.0,1.0,1.0,3.0,940.0,0.0,2.0,0.0,0.0187891,18 / 958
0.0,2.0,7.0,1.0,0.0,0.0,0.0,1016.0,1.0,1.0,0.0116732,"12 / 1,028"
1.0,2.0,2.0,3.0,0.0,1.0,0.0,2.0,959.0,4.0,0.0154004,15 / 974
1.0,0.0,0.0,2.0,6.0,4.0,0.0,2.0,4.0,990.0,0.0188305,"19 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.989
2,0.998
3,0.9994
4,0.9997
5,0.9998
6,0.9998
7,1.0
8,1.0
9,1.0
10,1.0
